# IFC to X3D, geometry and visual properties export only

In [ ]:
import os

import ifcopenshell, ifcopenshell.geom, ifcopenshell.util.element
from IPython.display import HTML
from OCC.Extend.DataExchange.X3D import X3DSceneExporter
from OCC.Display.WebGl.jupyter_renderer import create_download_link, progress_indicator
from OCC.Extend.DataExchange.x3d_standard import x3d as XX3D

In [ ]:
ifc_filename = './NBS_PremierLoftLadders_AlumLoftLddrs_Isotec.ifc'

In [ ]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)  # tells ifcopenshell to use pythonocc
settings.set(settings.DISABLE_TRIANGULATION, True)  # useless, since it is performed by the Tesselator
settings.set(settings.NO_NORMALS, True)

In [ ]:
ifc_file = ifcopenshell.open(ifc_filename)

In [ ]:
# create X3D scene
scene_exp = X3DSceneExporter()

In [ ]:
# traverse elements, create geometry is Representation is not None
elements = ifc_file.by_type("IfcElement")
idx_elt = 0
nb_elements = len(elements)
nb_failed = 0

for element in elements:
    if element.Representation is not None:  # some IfcProducts don't have any 3d representation
        try:
            elmnt_shape = ifcopenshell.geom.create_shape(settings, inst=element)
        except:
            nb_failed += 1
            continue
        
        if elmnt_shape.styles:
            r,g,b,alpha = elmnt_shape.styles[0] # the shape color
            color = (abs(r), abs(g), abs(b))
        else:  #default color, mid gray
            color = (0.5, 0.5, 0.5)

        out_node = scene_exp.add_shape(elmnt_shape.geometry,
                                       shape_color=color,
                                       emissive=False,
                                       export_edges=False)
    else:
        print(element.ObjectType, "has no representation")
    idx_elt +=1

    # update progress indicator
    progress_indicator(idx_elt / nb_elements)

In [ ]:
create_download_link(a_str=scene_exp.to_xml(), filename="ifc_stair.x3d")

In [ ]:
HTML(scene_exp.to_x3dom_html())